# INFO-F-208

## Partie 1.1

Avant d'implémenter un algorithme qui calcule l’alignement entre deux séquences, vous implémentez deux ADT (Abstract Data Type):    

* Un ADT séquence qui représente une séquence d’acides aminés et tous les opérations qu’on peut exécuter sur une séquence.

In [4]:
class Sequence():
    """
    ADT séquence qui représente une séquence d’acides aminés
    et tous les opérations qu’on peut exécuter sur une séquence.
    """
    def __init__(self, sequence):
        self.sequence = sequence
    
    def __repr__(self):
        return self.sequence
    
    def __len__(self):
        return len(self.sequence)
    
    def __getitem__(self, index):
        """
        @desc: Permet d'interpreter la classe comme une "String".
        
        @param{index}: Index de la lettre qu'on veut consulter.
        """
        return self.sequence[index]

    @staticmethod
    def load(filename):
        """
        """
        with open(filename) as f:
            seq = ""
            for line in f:
                if line[0] == ">":
                    if seq:
                        yield Sequence(seq)
                    seq = ""
                else:
                    seq += line[:-1] # Not including '\n'
                    
            if (seq):
                yield Sequence(seq)

In [5]:
sh3sequence = [seq for seq in Sequence.load("./SH3-sequence.fasta")]
maguk = [seq for seq in Sequence.load("./maguk-sequences.fasta")]

* Un ADT score qui représente une matrice de substitution et les opérations qu’on peut exécuter sur cette matrice.

In [6]:
class _ScoreLine:
    """
    ADT utilisée pour pouvoir utiliser l'ADT Score comme une matrice.
    """
    def __init__(self, line, indexes):
        self.line = line
        self.indexes = indexes
    
    def __getitem__(self, letter):
        j = self.indexes.index(letter)
        return self.line[j]

class Score:
    INDEXES = "ARNDCQEGHILKMFPSTWYV"
    def __init__(self, matrix = [[]], indexes = INDEXES):
        self.matrix = matrix
        self.indexes = indexes
        
    def __getitem__(self, letter): 
        """
        @desc: Renvoie le score d'une combinaison de deux acides
        
        @param{letter}: Le premier acide.
        @ex: Score['A']['Q']
        """
        i = self.indexes.index(letter)
        return _ScoreLine(self.matrix[i], self.indexes)

    def __repr__(self):
        ret = "  "
        for char in self.indexes:
            ret += " %3s " % (char)
        ret += '\n'
        i = 0
        for line in self.matrix:
            ret += "%3s " % (self.indexes[i])
            i += 1
            for char in line:
                if (char >= 0):
                    ret += ' '
                ret += "%3s " % (char)
            ret += '\n'

        return ret
    
    @staticmethod
    def load(filename):
        with open(filename) as f:
            matrix = []
            indexes = Score.INDEXES
            for line in f:
                if line[0] == ' ' or line[0] == '\t':
                    indexes = line.split()
                elif (line[0].isalpha()) or (line[0] == '*') or (line[0] == '-'):
                    l = line[1:]
                    matrix.append(list(map(int, l.split())))
                    
        return Score(matrix, indexes)

# Implémentation de l'algorithme de Needleman-Wunsch

En utilisant les ADT construits pendant l'étape précédente, vous implémentez en Python l’algorithme Needleman‐Wunsch qui calcule l’alignement global en utilisant la pénalité affine.

In [7]:
OPENING_GAP_PENALTY = -12
EXTENDING_GAP_PENALTY = -2

In [8]:
def init_matrix(x, y):
    """
    Initialize the matrix with zeros and a column of base score.
    """
    return [[(i * (OPENING_GAP_PENALTY + (j - 1) * EXTENDING_GAP_PENALTY)) + (j * (OPENING_GAP_PENALTY + (i - 1) * EXTENDING_GAP_PENALTY)) if (i == 0 or j == 0) else 0 for j in range(y + 1)] for i in range(x + 1)]

In [9]:
blosum80 = Score.load('./blosum80.txt')
blosum62 = Score.load('./blosum62.txt')
blosum40 = Score.load('./blosum40.txt')
pam120 = Score.load('./pam120.txt')
pam60 = Score.load('./pam60.txt')

In [10]:
class NeedlemanWunsch:
    def __init__(self, seq1, seq2, scoringMatrix):
        self.seq1 = seq1
        self.seq2 = seq2
        self.scoringMatrix = scoringMatrix
        self.result = []
        
        self.m = len(seq1)
        self.n = len(seq2)
        
        self.S = [[(i * (OPENING_GAP_PENALTY + (j - 1) * EXTENDING_GAP_PENALTY)) + (j * (OPENING_GAP_PENALTY + (i - 1) * EXTENDING_GAP_PENALTY)) if (i == 0 or j == 0) else 0 for j in range(self.n + 1)] for i in range(self.m + 1)]
        self.V = [[(i * (OPENING_GAP_PENALTY + (j - 1) * EXTENDING_GAP_PENALTY)) + (j * (OPENING_GAP_PENALTY + (i - 1) * EXTENDING_GAP_PENALTY)) if (i == 0 or j == 0) else 0 for j in range(self.n + 1)] for i in range(self.m + 1)]
        self.W = [[(i * (OPENING_GAP_PENALTY + (j - 1) * EXTENDING_GAP_PENALTY)) + (j * (OPENING_GAP_PENALTY + (i - 1) * EXTENDING_GAP_PENALTY)) if (i == 0 or j == 0) else 0 for j in range(self.n + 1)] for i in range(self.m + 1)]
        
        for i in range(1, self.m + 1):
            for j in range(1, self.n + 1):
                # Voir slide 63 (L3 alignement de séquences)
                self.V[i][j] = max(
                    self.S[i - 1][j] + OPENING_GAP_PENALTY + EXTENDING_GAP_PENALTY,
                    self.V[i - 1][j] + EXTENDING_GAP_PENALTY
                )

                self.W[i][j] = max(
                    self.S[i][j - 1] + OPENING_GAP_PENALTY + EXTENDING_GAP_PENALTY,
                    self.W[i][j - 1] + EXTENDING_GAP_PENALTY
                )
            
                self.S[i][j] = max(
                    self.S[i - 1][j - 1] + scoringMatrix[self.seq1[i - 1]][self.seq2[j - 1]],
                    self.V[i][j],
                    self.W[i][j] 
                ) 
    
        self._NW("", "", self.m, self.n)
        
    def __repr__(self):
        ret = ""
        for i in range(0, len(self.result)):
            ret += "%s\n" % (self.result[i][0]) # seq1
            identity = 0
            similarity = 0
            gap = 0
            
            for j in range(0, len(self.result[i][0])):
                a = self.result[i][0][j]
                b = self.result[i][1][j]
                if (a == b):
                    ret += ':'
                    identity += 1
                    similarity += 1
                elif (a != '-' and b != '-' and self.scoringMatrix[a][b] >= 0):
                    ret += '.'
                    similarity += 1
                else:
                    ret += ' '
                    gap += 1
            ret += "\n"
            ret += "%s\n" % (self.result[i][1]) # seq2
            ret += "%3.3f%% identity\n" % (100 * identity / len(self.result[i][0]))
            ret += "%3.3f%% similarity\n" % (100 * similarity / len(self.result[i][0]))
            ret += "%3.3f%% gap\n"  % (100 * gap / len(self.result[i][0]))
            ret += "Length : %i\n" % (len(self.result[i][0]))
            ret += "Global score : %s\n" % (self.S[self.m][self.n])
        return ret

        
    def _NW(self, align1, align2, i, j):
        """
        """
        if (i > 0 and j > 0) and (self.S[i][j] == self.S[i - 1][j - 1] + self.scoringMatrix[self.seq1[i - 1]][self.seq2[j - 1]]):
            self._NW(self.seq1[i - 1] + align1, self.seq2[j - 1] + align2, i - 1, j - 1)
        elif (i > 0) and (self.S[i][j] == self.V[i][j]):
            self._NW(self.seq1[i - 1] + align1, "-" + align2, i - 1, j)
        elif (j > 0) and (self.S[i][j] == self.W[i][j]):
            self._NW("-" + align1, self.seq2[j - 1] + align2, i, j - 1)
        else:
            # end of backtracking : we are back in S[0][0]
            self.result.append((align1, align2))

Vérifiez si votre programme donne le même résultat que l’outil LALIGN http://www.ch.embnet.org/software/LALIGN_form.html

Utilisez les séquences dans le fichier SH3-sequences.fasta sur le site web.        

In [11]:
print(NeedlemanWunsch(sh3sequence[0], sh3sequence[1], blosum80))

GGVTTFVALYDYESRTETDLSFKKGERLQIVNNTEGDWWLAHSLSTGQTGYIPSNYVAPSDS
      .: ::... .. .::::.:. :...:      :   .: .:. :.:: ::.     
-M--EAIAKYDFKATADDELSFKRGDILKVLNEECDQNWYKAEL-NGKDGFIPKNYIEMKPH
29.032% identity
56.452% similarity
43.548% gap
Length : 62
Global score : 59



Le résultat avec _LALIGN_ est le suivant:

-----------------------------------------------------------------------------------------

```
lobal/global (N-W) score: 114; 30.2% identity (61.9% similar) in 63 aa overlap (1-60:1-58)

               10        20        30         40        50          60         
unknow GGVTTFVALYDYESRTETDLSFKKGERLQIVNNT-EGDWWLAHSLSTGQTGYIPSNYV--APS
             .: ::... .. .::::.:. :...:   . .:. :.   .:. :.:: ::.   :
unknow ---MEAIAKYDFKATADDELSFKRGDILKVLNEECDQNWYKAEL--NGKDGFIPKNYIEMKPH
                  10        20        30        40          50     
```

-----------------------------------------------------------------------------------------

In [12]:
print(NeedlemanWunsch(sh3sequence[0], sh3sequence[1], pam120))

GGVTTFVALYDYESRTETDLSFKKGERLQIVNNTEGDWWLAHSLSTGQTGYIPSNYVAPSDS
    . .: ::. . .. .::::.:. :...:.  .. :    :. :. :.:: ::..    
-M-EA-IAKYDFKATADDELSFKRGDILKVLNEECDQNWYKAELN-GKDGFIPKNYIEMKPH
29.032% identity
61.290% similarity
38.710% gap
Length : 62
Global score : 57



### Résultat de _LALIGN_

-----------------------------------------------------------------------

```
 n-w opt:  57  Z-score: 157.8  bits: 32.6 E(1): 2.1e-27
global/global (N-W) score: 57; 29.0% identity (61.3% similar) in 62 aa overlap (1-62:1-58)

               10        20        30        40        50        60
unknow GGVTTFVALYDYESRTETDLSFKKGERLQIVNNTEGDWWLAHSLSTGQTGYIPSNYVAPSDS
         .   .: ::. . .. .::::.:. :...:.  .. :    :. :. :.:: ::..  
unknow MEA---IAKYDFKATADDELSFKRGDILKVLNEECDQNWYKAELN-GKDGFIPKNYIEMKPH
                  10        20        30        40         50               
```

-----------------------------------------------------------------------

Ici en utilisant la matrice de score PAM120 l'algorithme donne le même résultat que _LALIGN_ tfa

## Partie 1.3

Modifiez le logiciel de la partie 1.2 de sorte que on peut faire un alignement local (Smith‐Waterman). Utilisez les séquences dans le fichier maguk-sequences.fasta sur le site web. 

In [13]:
class SmithWaterman:
    def __init__(self, seq1, seq2, scoringMatrix):
        self.seq1 = seq1
        self.seq2 = seq2
        self.scoringMatrix = scoringMatrix
        self.result = []
        
        self.m = len(seq1)
        self.n = len(seq2)
        
        self.max = [self.m, self.n]
        
        self.S = init_matrix(self.m, self.n)
        self.V = init_matrix(self.m, self.n)
        self.W = init_matrix(self.m, self.n)
        
        # Remplit les matrices en utilisant les formules vue dans les slides du cour.
        for i in range(1, self.m + 1):
            for j in range(1, self.n + 1):
                # Voir slide 63 (L3 alignement de séquences)
                self.V[i][j] = max(
                    self.S[i - 1][j] + OPENING_GAP_PENALTY + EXTENDING_GAP_PENALTY,
                    self.V[i - 1][j] + EXTENDING_GAP_PENALTY
                )

                self.W[i][j] = max(
                    self.S[i][j - 1] + OPENING_GAP_PENALTY + EXTENDING_GAP_PENALTY, 
                    self.W[i][j - 1] + EXTENDING_GAP_PENALTY
                )
            
                self.S[i][j] = max(
                    self.S[i - 1][j - 1] + scoringMatrix[self.seq1[i - 1]][self.seq2[j - 1]],
                    self.V[i][j],
                    self.W[i][j],
                    0
                ) 

                if (self.S[i][j] > self.S[self.max[0]][self.max[1]]):
                    self.max = [i, j]
    
        self._SW("", "", self.m, self.n)
        
    def __repr__(self):
        ret = ""
        for i in range(0, len(self.result)):
            ret += "%s\n" % (self.result[i][0]) # seq1
            identity = 0
            similarity = 0
            gap = 0
            for j in range(0, len(self.result[i][0])):
                a = self.result[i][0][j]
                b = self.result[i][1][j]
                if (a == b):
                    ret += ':'
                    identity += 1
                    similarity += 1
                elif (a != '-' and b != '-' and self.scoringMatrix[a][b] >= 0):
                    ret += '.'
                    similarity += 1
                else:
                    ret += ' '
                    gap += 1
            ret += "\n"
            ret += "%s\n" % (self.result[i][1]) # seq2
            ret += "%3.3f%% identity\n" % (100 * identity / len(self.result[i][0]))
            ret += "%3.3f%% similarity\n" % (100 * similarity / len(self.result[i][0]))
            ret += "%3.3f%% gap\n"  % (100 * gap / len(self.result[i][0]))
            ret += "Length : %i\n" % (len(self.result[i][0]))
            ret += "Global score : %s\n" % (self.S[self.max[0]][self.max[1]])
        return ret

        
    def _SW(self, align1, align2, i, j):
        """
        """
        # print("---\nALIGN1: %s\nALIGN2: %s\n---" % (align1, align2))
        if (i > 0 and j > 0) and (self.S[i][j] > 0):
            if self.S[i][j] == self.S[i - 1][j - 1] + self.scoringMatrix[self.seq1[i - 1]][self.seq2[j - 1]]:
                # Vérification par apport à la diagonale.
                self._SW(self.seq1[i - 1] + align1, self.seq2[j - 1] + align2, i - 1, j - 1)
            elif self.S[i][j] == self.V[i][j]:
                # Vérification par apport à la gauche: trous dans seq2
                self._SW(self.seq1[i - 1] + align1, "-" + align2, i - 1, j)
            elif self.S[i][j] == self.W[i][j]:
                # Vérification par apport au dessus: trous sequ1
                self._SW("-" + align1, self.seq2[j - 1] + align2, i, j - 1)

        else:
            self.result.append((align1, align2))

In [14]:
import sys
sys.setrecursionlimit(10000)

print(SmithWaterman(maguk[0], maguk[1], pam120))


VNGTDADYEYEEITLERGNSGLGFSIAGGTDNPHIGDDSSIFITKIITGGAAAQDGRLRVNDCILRVNEVDVRDVTHSKAVEALKEAGSIVRLYVKRRKPVSEKIMEIKLIKGPKGLGFSIAGGVGNQHIPGDNSIYVTKIIEGGAAHKDGKLQIGDKLLAVNNVCLEEVTHEEAVTALKNTSDFVYLKVAKPTSMYMNDGYAPPDITNSSSQPVDNHVSP-SS--FLG----------QTPASPARYSPVSKAVLGDDEITREPRKVVLHRGSTGLGFNIVGGEDGEGIFISFILAGGPADLSGELRKGDRIISVNSVDLRAASHEQAAAALKNAGQAVTIVAQYRPEEYSRFEAKIHDLREQMMNSSISSGSGSLRTSQKRSLYVRALFDYDKTKDSGLPSQGLNFKFGDILHVINASDDEWWQARQVTPDGESDEVGVIPSKRRVEKKERARLKTVKFNSKTRDKGEIPDDMGSKGLKHVTSNASDSESSYRGQEEYVLSYEPVNQQEVNYTRPVIILGPMKDRINDDLISEFPDKFGSCVPHTTRPKRDYEVDGRDYHFVTSREQMEKDIQEHKFIEAGQYNNHLYGTSVQSVREVAEKGKHCILDVSGNAIKRLQIAQLYPISIFIKPKSMENIMEMNKRLTEEQARKTFERAMKLEQEFTEHFTAIVQGDTLEDIYNQVKQIIEEQSGSYIWVPAKEKL
:::.:. . ::::.:::::::::::::::.::::. ::..::::::: ::::: :::: ::::.:::::::: .:.::.:::::::::..::: :.::.: .: :::..:.:::::::::::::.::::::::::::.:::::::::.:::.:::::.::::::. :..: :::::..:::::: :::::::: :. .:: ::::: ... .  .:::.:  ::  .::          . ...:.::::... .:.....::::::..::.:::::::::::::::::::.::

### Résultat de LALIGN

Identique en utilisant PAM120.

```
 Waterman-Eggert score: 2711;  1234.7 bits; E(1) <  0
73.0% identity (90.2% similar) in 705 aa overlap (213-904:120-817)

            220       230       240       250       260       270  
unknow VNGTDADYEYEEITLERGNSGLGFSIAGGTDNPHIGDDSSIFITKIITGGAAAQDGRLRV
       :::.:. . ::::.:::::::::::::::.::::. ::..::::::: ::::: :::: :
unknow VNGSDGMFKYEEIVLERGNSGLGFSIAGGIDNPHVPDDPGIFITKIIPGGAAAMDGRLGV
     120       130       140       150       160       170         

            280       290       300       310       320       330  
unknow NDCILRVNEVDVRDVTHSKAVEALKEAGSIVRLYVKRRKPVSEKIMEIKLIKGPKGLGFS
       :::.:::::::: .:.::.:::::::::..::: :.::.: .: :::..:.:::::::::
unknow NDCVLRVNEVDVSEVVHSRAVEALKEAGPVVRLVVRRRQPPPETIMEVNLLKGPKGLGFS
     180       190       200       210       220       230         

            340       350       360       370       380       390  
unknow IAGGVGNQHIPGDNSIYVTKIIEGGAAHKDGKLQIGDKLLAVNNVCLEEVTHEEAVTALK
       ::::.::::::::::::.:::::::::.:::.:::::.::::::. :..: :::::..::
unknow IAGGIGNQHIPGDNSIYITKIIEGGAAQKDGRLQIGDRLLAVNNTNLQDVRHEEAVASLK
     240       250       260       270       280       290         

            400       410       420       430                      
unknow NTSDFVYLKVAKPTSMYMNDGYAPPDITNSSSQPVDNHVSPSSFLG-------------Q
       :::: :::::::: :. .:: ::::: ... .  .:::.: .: ::             .
unknow NTSDMVYLKVAKPGSLHLNDMYAPPDYASTFTALADNHISHNSSLGYLGAVESKVSYPAP
     300       310       320       330       340       350         

     440       450       460       470       480       490         
unknow TPASPARYSPVSKAVLGDDEITREPRKVVLHRGSTGLGFNIVGGEDGEGIFISFILAGGP
        ...:.::::... .:.....::::::..::.:::::::::::::::::::.::::::::
unknow PQVPPTRYSPIPRHMLAEEDFTREPRKIILHKGSTGLGFNIVGGEDGEGIFVSFILAGGP
     360       370       380       390       400       410         

     500       510       520       530       540       550         
unknow ADLSGELRKGDRIISVNSVDLRAASHEQAAAALKNAGQAVTIVAQYRPEEYSRFEAKIHD
       ::::::::.::::.:::.:.:: :.::::::::: :::.::::::::::::::::.::::
unknow ADLSGELRRGDRILSVNGVNLRNATHEQAAAALKRAGQSVTIVAQYRPEEYSRFESKIHD
     420       430       440       450       460       470         

     560       570       580       590       600       610         
unknow LREQMMNSSISSGSGSLRTSQKRSLYVRALFDYDKTKDSGLPSQGLNFKFGDILHVINAS
       :::::::::.::::::::::.:::::::::::::.:.:: ::::::.: .::::::::::
unknow LREQMMNSSMSSGSGSLRTSEKRSLYVRALFDYDRTRDSCLPSQGLSFSYGDILHVINAS
     480       490       500       510       520       530         

     620       630       640       650       660       670         
unknow DDEWWQARQVTPDGESDEVGVIPSKRRVEKKERARLKTVKFNSKTRDKGEIPDDMGSKGL
       :::::::: :::.:::...::::::.:::::::::::::::...:   : : .. .  ::
unknow DDEWWQARLVTPHGESEQIGVIPSKKRVEKKERARLKTVKFHART---GMIESNRDFPGL
     540       550       560       570       580          590      

     680       690       700       710       720       730         
unknow KHVTSNASDSESSYRGQEEYVLSYEPVNQQEVNYTRPVIILGPMKDRINDDLISEFPDKF
           :..  .. . .:::. .::::::..::..:.::::::::::::.:::::::::.::
unknow ----SDDYYGAKNLKGQEDAILSYEPVTRQEIHYARPVIILGPMKDRVNDDLISEFPHKF
            600       610       620       630       640       650  

     740       750       760       770       780       790         
unknow GSCVPHTTRPKRDYEVDGRDYHFVTSREQMEKDIQEHKFIEAGQYNNHLYGTSVQSVREV
       ::::::::::.:: ::::.:::::.::::::::::..:::::::.:..:::::.::::.:
unknow GSCVPHTTRPRRDNEVDGQDYHFVVSREQMEKDIQDNKFIEAGQFNDNLYGTSIQSVRAV
            660       670       680       690       700       710  

     800       810       820       830       840       850         
unknow AEKGKHCILDVSGNAIKRLQIAQLYPISIFIKPKSMENIMEMNKRLTEEQARKTFERAMK
       ::.::::::::::::::::: ::::::.:::::::.: .::::.: : ::: :....:::
unknow AERGKHCILDVSGNAIKRLQQAQLYPIAIFIKPKSIEALMEMNRRQTYEQANKIYDKAMK
            720       730       740       750       760       770  

     860       870       880       890       900    
unknow LEQEFTEHFTAIVQGDTLEDIYNQVKQIIEEQSGSYIWVPAKEKL
       ::::: : ::::::::.::.:::..:::::.::: :::::. :::
unknow LEQEFGEYFTAIVQGDSLEEIYNKIKQIIEDQSGHYIWVPSPEKL
            780       790       800       810       
```

Retrouvez les similarités entre les 4 séquences. Expliquez les similarités. Plus d’informations concernant les protéines dans le fichier .fasta peuvent être trouvées sur le site UniProt (http://www.uniprot.org/).

Le résultat du site uniprot est le suivant: http://www.uniprot.org/align/A20161102F725F458AC8690F874DD868E4ED79B8896EBCFO .
Les similaritées sont dû à l'origine humaine des protéines.